In [4]:
import os

os.chdir("..")



In [5]:
pwd

'd:\\projects\\MLflow_DL\\Machine-learning-project-with-MLflow'

fix add to our config first and then we create an entity here

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir:Path
    data_path:Path


In [9]:
# now we need to create configuration manager function for data transformation task and for that we need the constants that store paths to config yaml schema and params and also read yaml and create directories functions to read them from utils

from mlProject.utils.common import read_yaml, create_directories
from mlProject.constants import *





In [21]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.schema = read_yaml(schema_filepath)
            self.params = read_yaml(params_filepath)
            print(self.config)

            create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
          config = self.config.data_transformation

          create_directories([config.root_dir])

          data_transformation_config = DataTransformationConfig(
                root_dir = config.root_dir,
                data_path = config.data_path,
          )
          return data_transformation_config
          

In [22]:
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

here we can apply data cleaning, eda and other things but for now the data is already clean and we will only be focussing on the pipeline so lets move forward

In [23]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config


    def data_split(self):
        data = pd.read_csv(self.config.data_path, sep=";")

        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index = False)

        logger.info("Data splitted into train and test sets")
        logger.info(f"Train shape: {train.shape}, Test shape: {test.shape}")


now lets set the pipeline

In [24]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.data_split()
except Exception as e:
    raise e

[2025-01-23 12:24:21,864: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-23 12:24:21,872: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-23 12:24:21,880: INFO: common: yaml file: params.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://raw.githubusercontent.com/maymunashah/Machine-learning-project-with-MLflow/main/wine+quality.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/winequality-red.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}}
[2025-01-23 12:24:21,888: INFO: common: created directory at: artifacts]
[2025-01-23 12:24:21,888: INFO: common: cre